In [12]:
import numpy as np
import pandas as pd
import os
import shutil
from subprocess import Popen, PIPE
from IPython.display import clear_output
import time

In [22]:
PYPATH = '/home/shibal/anaconda3/envs/jasa/bin/python'
FILEPATH = '/home/shibal/Additive-Models-with-Structured-Interactions/SparseAMsWithInteractions/src/AMsWithInteractionsL0/AMsWithInteractionsL0-Synthetic.py'
PATH = "/pool001/shibal/results-synthetic"

version = 14
# n = 100
r = 1.0
dist = 'heteroskedastic'

In [23]:
def make_bash_file(seed, version, dist, r):
    bash_folder_path = f"{PATH}/bashes/{dist}/v{version}_r{r}"
    bash_file_path = os.path.join(bash_folder_path, "seed{}.sh".format(seed))
    log_path = f"{PATH}/logs/{dist}/v{version}_r{r}/seed{seed}"
    os.makedirs(bash_folder_path,exist_ok=True)
    os.makedirs(log_path,exist_ok=True)
    with open(bash_file_path,"w") as f:
        f.write("#!/bin/bash\n")
        f.write("#SBATCH --cpus-per-task=1\n")
        f.write("#SBATCH --time=1-00:00\n")
        f.write("#SBATCH --mem=8G\n")
        f.write("#SBATCH -p sched_mit_sloan_batch\n")        
        f.write("#SBATCH --mail-type=FAIL\n")
        f.write("#SBATCH --mail-user=shibal@mit.edu\n")
        f.write(f"#SBATCH -o {log_path}/seed{seed}_%j.out\n")
        f.write(f"#SBATCH -e {log_path}/_seed{seed}_%j.err\n\n")
        f.write("module load sloan/python/modules/python-3.6/gurobipy/9.0.1\n\n")
        f.write(f"{PYPATH} -u {FILEPATH}  --dist {dist} --seed {seed} --train_size {100} --version {version} --r {r} |& tee -a {log_path}/output.txt\n\n") 
        f.write(f"{PYPATH} -u {FILEPATH}  --dist {dist} --seed {seed} --train_size {200} --version {version} --r {r} |& tee -a {log_path}/output.txt\n\n") 
        f.write(f"{PYPATH} -u {FILEPATH}  --dist {dist} --seed {seed} --train_size {400} --version {version} --r {r} |& tee -a {log_path}/output.txt\n\n") 
#         f.write(f"{PYPATH} -u {FILEPATH}  --dist {dist} --seed {seed} --train_size {1000} --version {version} --r {r} |& tee -a {log_path}/output.txt\n\n") 
    return bash_file_path


In [24]:
bash_files = []
seeds = np.arange(100)
for seed in seeds:
    bash_files.append(make_bash_file(seed, version, dist, r))

In [25]:
torun = seeds
# torun = range(1,6)
submitted = []
print(len(torun))

100


In [26]:
exit_code = 1
for i, seed in enumerate(seeds):
    if i % 100 == 0:
        clear_output(wait=True)
    print(i)
    sh = make_bash_file(seed, version, dist, r)
    while True:
        process = Popen(["sbatch",sh], stdout=PIPE)
        (output, err) = process.communicate()
        exit_code = process.wait()
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())),output,err)
        if exit_code == 0:
            print(sh,"submitted!")
            tmp_id = str(output)[-11:-3]
            print("job id:", tmp_id)
            submitted.append(tmp_id)
            break
        time.sleep(10000)

0
2023-02-21 23:14:50 b'Submitted batch job 46989171\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed0.sh submitted!
job id: 46989171
1
2023-02-21 23:14:50 b'Submitted batch job 46989172\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed1.sh submitted!
job id: 46989172
2
2023-02-21 23:14:50 b'Submitted batch job 46989173\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed2.sh submitted!
job id: 46989173
3
2023-02-21 23:14:51 b'Submitted batch job 46989174\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed3.sh submitted!
job id: 46989174
4
2023-02-21 23:14:51 b'Submitted batch job 46989175\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed4.sh submitted!
job id: 46989175
5
2023-02-21 23:14:51 b'Submitted batch job 46989176\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed5.sh submitted!
job id: 46989176
6
2023-02-21 23:

2023-02-21 23:15:02 b'Submitted batch job 46989221\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed50.sh submitted!
job id: 46989221
51
2023-02-21 23:15:02 b'Submitted batch job 46989222\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed51.sh submitted!
job id: 46989222
52
2023-02-21 23:15:02 b'Submitted batch job 46989223\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed52.sh submitted!
job id: 46989223
53
2023-02-21 23:15:02 b'Submitted batch job 46989224\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed53.sh submitted!
job id: 46989224
54
2023-02-21 23:15:03 b'Submitted batch job 46989225\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed54.sh submitted!
job id: 46989225
55
2023-02-21 23:15:03 b'Submitted batch job 46989226\n' None
/pool001/shibal/results-synthetic/bashes/heteroskedastic/v14_r1.0/seed55.sh submitted!
job id: 46989226
56
2023

In [21]:
from subprocess import Popen, PIPE

In [15]:
for job in range(45764232, 45764334):
    process = Popen(['scancel',str(job)], stdout=PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait()
    if exit_code ==0:
        print(job, "deleted!")

45764232 deleted!
45764233 deleted!
45764234 deleted!
45764235 deleted!
45764236 deleted!
45764237 deleted!
45764238 deleted!
45764239 deleted!
45764240 deleted!
45764241 deleted!
45764242 deleted!
45764243 deleted!
45764244 deleted!
45764245 deleted!
45764246 deleted!
45764247 deleted!
45764248 deleted!
45764249 deleted!
45764250 deleted!
45764251 deleted!
45764252 deleted!
45764253 deleted!
45764254 deleted!
45764255 deleted!
45764257 deleted!
45764258 deleted!
45764260 deleted!
45764261 deleted!
45764262 deleted!
45764263 deleted!
45764264 deleted!
45764265 deleted!
45764266 deleted!
45764267 deleted!
45764268 deleted!
45764269 deleted!
45764270 deleted!
45764271 deleted!
45764272 deleted!
45764273 deleted!
45764274 deleted!
45764275 deleted!
45764276 deleted!
45764277 deleted!
45764278 deleted!
45764279 deleted!
45764280 deleted!
45764281 deleted!
45764282 deleted!
45764283 deleted!
45764284 deleted!
45764285 deleted!
45764286 deleted!
45764287 deleted!
45764288 deleted!
45764289 d

In [33]:
for seed in range(100):
    command = f"""rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed{seed}/AMsWithInteractionsL0/v13/r1.0"""
    print(command)
    !{command}

rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed0/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed1/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed2/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed3/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed4/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed5/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed6/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed7/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed8/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic

rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed78/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed79/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed80/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed81/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed82/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed83/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed84/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed85/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/heteroskedastic/N_train_400/seed86/AMsWithInteractionsL0/v13/r1.0
rm -r /pool001/shibal/results-synthetic/hetero